In [ ]:
# Imports
#%matplotlib widget
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
import h5py
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from diffpy.pdfgetx.pdfconfig import PDFConfig
from diffpy.pdfgetx.pdfgetter import PDFGetter
from diffpy.morph.morphpy import morph_arrays
from diffpy.utils.parsers import load_data
from ufpdf_xfel_scripts.lcls.paths import synchrotron_data_dir, experiment_data_dir, global_output_dir
from ufpdf_xfel_scripts.lcls.run import Run
from ufpdf_xfel_scripts.lcls.plots import plot_delay_scans, plot_static_scans

import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("diffpy.pdfgetx").setLevel(logging.ERROR)
logging.getLogger("diffpy.pdfgetx.user").setLevel(logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
# Experiment: LCLS 202601
instrument = 'mfx'
experiment_number = 'l1044925'

In [ ]:
# User: Luis
username = 'lkitsu'

In [ ]:
# Setup:

# setup for roi
q_min =2.5 # I(q) figure of merit (fom)
q_max = 3 # I(q) fom
r_min_fom = 2 # G(r) fom 
r_max_fom = 5 # G(r) fom

# global setup for morphs
target_id = 0 #index of the delay_time Iq_off to use as target for morphing as well as calculate adhoc corrections for g(r)

# setup for morph I(q) delay
q_min_morph = 0 #qmin for normalization. None uses the smaller value. Not working for new diffpy.morph! Also is called xmin now.
q_max_morph = 12 #qmax for normalization. None uses the largest value. Not working for new diffpy.morph! Also is called xmax now.
scale = 1.01 
stretch = None
smear = None
baselineslope = None
qdamp = None
params = {'xmin': q_min_morph, 'xmax': q_max_morph, 'scale': scale,'stretch': stretch, 'smear':smear}
t0 = 0
points_away_t0_plot_on_off = 0

# setup for morph F(q)
fit_qmin = 0.1
fit_qmax = 10
initial_guesses = {'rpoly': 1, 'qmin': 0.1, 'bgscale': 1}
squeeze = [0,0,0,0,0]

In [ ]:
# Run:
run_number = 58
background_number = 1
sample_name = 'NSPSe'
sample_composition = 'Na11SnPSe12'
number_of_static_samples = 11   # only needed for static runs, will be ignored otherwise
verbose = False
plot_monitor_normalizations = False
# delay motor for run
delay_motor = "mfx_lxt_fast2"
# delay_motor = "lxt_ttx"

In [ ]:
# Instantiate run object
run_object = Run(run_number,
                 background_number,
                 sample_name,
                 sample_composition,
                 instrument,
                 experiment_number,
                 number_of_static_samples=number_of_static_samples,
                 delay_motor=delay_motor,
                 )
morph_parameters = run_object.morph_parameters # Dictionary with fitted morph parameters. The key is the delay.

In [ ]:
# Plot control
if run_object.delay_scan:
    plot_delay_scans(run_object.morphed_delay_scans, run_object)
    if plot_monitor_normalizations:
        plot_delay_scans(run_object.mon1_normalized_delay_scans, run_object)
        plot_delay_scans(run_object.mon2_normalized_delay_scans, run_object)
else:
    plot_static_scans(run_object.morphed_static_scans, run_object)



In [ ]:
def pdfgetter_function(x, y, rpoly, qmin, bgscale):
    """
    Transform input I(Q) to F(Q) using PDFGetter with background correction.
    Returns (x, F(Q)).
    """
    cfg = PDFConfig()
    cfg.composition = sample_composition
    cfg.rpoly = rpoly
    cfg.qmin = qmin
    cfg.qmax = 20
    cfg.qmaxinst = 20
    cfg.dataformat = 'QA'
    cfg.mode = 'xray'

    background_path = Background_path
    if background_path.exists():
        bg_data = np.loadtxt(background_path)
        if bg_data.ndim == 2 and bg_data.shape[1] == 2:
            bg_interp = np.interp(x, bg_data[:, 0], bg_data[:, 1])
        else:
            bg_interp = bg_data  # already on same grid
        cfg.background = bg_interp
        cfg.bgscale = bgscale
    else:
        print(f"Background file not found: {background_path}")
        cfg.background = None
        cfg.bgscale = 0.0

    # Run PDFGetter
    pg = PDFGetter(cfg)
    _, _ = pg(x=x, y=y)
    q_out, fq_out = pg.fq

    fq_interp = np.interp(x, q_out, fq_out)
    return x, fq_interp

In [ ]:
reference_delay = delays[target_id]   

x_morph = morph_delays[reference_delay][0]
y_morph = morph_delays[reference_delay][2]
x_target = q_synchrotron
y_target = fq_synchrotron

morph_table = np.column_stack([x_morph, y_morph])
target_table = np.column_stack([x_target, y_target])

morph_info, morph_result = morph_arrays(
    morph_table,
    target_table,
    funcxy=(pdfgetter_function, initial_guesses),
    scale=1,
    squeeze=squeeze,
    xmin=fit_qmin,
    xmax=fit_qmax
)

fitted_funcxy = morph_info['funcxy']
scale_opt = float(morph_info['scale'])
squeeze_opt = [float(v) for v in morph_info['squeeze'].values()]

In [ ]:
def compute_gr(q, fq, r_min=0, r_max=30, npoints=1000):
    r = np.linspace(r_min, r_max, npoints)
    qr = np.outer(q, r)
    integrand = fq[:, None] * np.sin(qr)
    gr = (2/np.pi) * np.trapezoid(integrand, q, axis=0)
    return r, gr

def plot_reference_comparison(q_target, fq_target,
                              q_morph, fq_morph,
                              r_min=0, r_max=30,
                              r_min_fom=None, r_max_fom=None):

    r, gr_target = compute_gr(q_target, fq_target)
    r, gr_morph = compute_gr(q_morph, fq_morph)

    fig, (ax0, ax1) = plt.subplots(2,1, figsize=(8,6))

    ax0.plot(q_target, fq_target, linestyle='--', label="Synchrotron F(Q)")
    ax0.plot(q_morph, fq_morph, label="XFEL F(Q)")
    ax0.set_xlabel("Q (1/Å)")
    ax0.set_ylabel("F(Q)")
    ax0.set_title("Reference: F(Q)")
    ax0.legend()

    ax1.plot(r, gr_target, label="G(r) Synchrotron", color='orange')
    ax1.plot(r, gr_morph, label="G(r) XFEL", color='black')

    if r_min_fom is not None:
        ax1.axvline(r_min_fom, color='red')
    if r_max_fom is not None:
        ax1.axvline(r_max_fom, color='red')

    ax1.set_xlabel("r (Å)")
    ax1.set_ylabel("G(r)")
    ax1.legend()

    plt.tight_layout()
    plt.show()


def plot_gr_function(gr_delay_dict,
                     sample_name,
                     run_number,
                     r_min_fom=None,
                     r_max_fom=None):

    fig, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(5,1, figsize=(8,14))

    delay_keys = sorted(gr_delay_dict.keys())
    cmap = matplotlib.colormaps['viridis']
    norm = plt.Normalize(min(delay_keys), max(delay_keys))

    for delay_t in delay_keys:
        pdata = gr_delay_dict[delay_t]
        color = cmap(norm(delay_t))

        ax0.plot(pdata["r"], pdata["gr_on"], color=color)
        ax1.plot(pdata["r"], pdata["gr_off"], color=color)
        ax2.plot(pdata["r"], pdata["diff_gr"], color=color)

    # ---- Integration window lines ----
    if r_min_fom is not None:
        ax2.axvline(r_min_fom, color='red')
    if r_max_fom is not None:
        ax2.axvline(r_max_fom, color='red')

    ax0.set_title(f"sample = {sample_name}, run = {run_number}")
    ax0.set_ylabel("G(r) ON")
    ax1.set_ylabel("G(r) OFF")
    ax2.set_ylabel("ΔG(r)")

    # ---- Delay metrics ----
    delay_times = delay_keys
    sum_on = [gr_delay_dict[d]["sum_gr_on"] for d in delay_times]
    sum_off = [gr_delay_dict[d]["sum_gr_off"] for d in delay_times]
    RMS = [gr_delay_dict[d]["RMS"] for d in delay_times]
    diff_int = [gr_delay_dict[d]["diff_int"] for d in delay_times]

    ax3.plot(delay_times, sum_on, marker='o', label='Pump ON')
    ax3.plot(delay_times, sum_off, marker='o', label='Pump OFF')
    ax3.set_ylabel("Integrated G(r)")
    ax3.legend(frameon=False)

    ax4.plot(delay_times, RMS, marker='o', label='RMS')
    ax4.plot(delay_times, diff_int, marker='o', label='Integral ΔG(r)')
    ax4.set_xlabel("Delay time")
    ax4.set_ylabel("Metric")
    ax4.legend(frameon=False)

    plt.tight_layout()
    plt.show()



In [ ]:
gr_delay_dict = {}

for delay_t in delay:

    q_iq  = morph_delays[delay_t][0]
    on_iq = morph_delays[delay_t][1]
    off_iq = morph_delays[delay_t][2]

    table_on  = np.column_stack([q_iq, on_iq])
    table_off = np.column_stack([q_iq, off_iq])

    target_dummy = table_on  # only to preserve grid

    _, fq_on_table = morph_arrays(
        table_on,
        target_dummy,
        funcxy=(pdfgetter_function, fitted_funcxy),
        scale=scale_opt,
        squeeze=squeeze_opt,
        xmin=fit_qmin,
        xmax=fit_qmax,
        apply=True
    )

    _, fq_off_table = morph_arrays(
        table_off,
        target_dummy,
        funcxy=(pdfgetter_function, fitted_funcxy),
        scale=scale_opt,
        squeeze=squeeze_opt,
        xmin=fit_qmin,
        xmax=fit_qmax,
        apply=True
    )

    q_final = fq_on_table[:,0]
    fq_on   = fq_on_table[:,1]
    fq_off  = fq_off_table[:,1]

    r, gr_on  = compute_gr(q_final, fq_on)
    r, gr_off = compute_gr(q_final, fq_off)

    diff = gr_on - gr_off

    rmin_idx = np.abs(r - r_min_fom).argmin()
    rmax_idx = np.abs(r - r_max_fom).argmin()

    gr_delay_dict[delay_t] = {
        "r": r,
        "gr_on": gr_on,
        "gr_off": gr_off,
        "diff_gr": diff,
        "RMS": np.sqrt(np.sum(diff[rmin_idx:rmax_idx]**2)),
        "diff_int": np.sum(diff[rmin_idx:rmax_idx]),
        "sum_gr_on": np.sum(gr_on[rmin_idx:rmax_idx]),
        "sum_gr_off": np.sum(gr_off[rmin_idx:rmax_idx]),
    }


In [ ]:
plot_reference_comparison(
    q_target=q_synchrotron,
    fq_target=fq_synchrotron,
    q_morph=morph_result[:, 0],
    fq_morph=morph_result[:, 1],
    r_min=0,
    r_max=30,
)

In [ ]:
plot_gr_function(
    gr_delay_dict,
    sample_name=sample_name,
    run_number=run_number,
    r_min_fom=r_min_fom,
    r_max_fom=r_max_fom
)